In [ ]:
knitr::opts_chunk$set(echo = TRUE,warning=TRUE)



# Introduction

Nous avions écrit un document pour montrer comment ajouter un revenu à une population synthétique, à partir de données aggrégées fournies par des déciles. Les relecteurs de Jasss n'ont pas vu que cette méthode pouvait s'appliquer dans des cas plus simples. Ce document présente  un cas " jouet " où la variable ajoutée est celle de la motorisation : c'est une variable binaire, plus simple que les revenus fournis sous forme de décomes/

Ce document reprend le document "Comparaison de deux Méthodologies pour attribuer un revenu aux ménages" en l'adaptant à ce nouveau cas d'étude plus simple.


Règles d'écriture :

* les noms des scalaires sont en minuscule
* les noms des objets de dimensions supérieurs commencent par une majuscule
* les noms des fonctions commencent par deux majuscules

# Le cas d'étude
Nous générons une population synthétique avec trois catégories qualitatives age de la personne de référence,  taille du ménage et ménage motorisé ou pas. La taille de la population est n (n est un multiple de 100 000)

Nous nous plaçons dans un cas où nous avons une population synthétique sans le taux de motorisation. Nous souhaitons ajouter à cette population synthétique la variable taux de motorisation en utilisant les données agrégées sur le taux motorisation par modalité d'age.

Par exemple, en connassant les informations suivantes 26 % des moins de 30 ans sont motorisé, 87% des 30-6à ans et 77% des plus de 60 ans, comment ajouter un taux de motorisation à une population synthétique qui comprend deux catégories age et taille du ménage ?


Pour cela, nous utilisons l'algorithme de maximisation de l'entropie tel qu'il est décrit dans notre article soumis à Jasss.


L'objectif de la méthodologie est de retrouver la variable motorisée  à partir de la donnée de la population synthétique sans motorisation et les données agréggées de la motorisation pour chaque modalité de l'age


Voici ce les étapes de ce travail

* Nous construisons un tableau de contingence qui croisent toutes les modalités des trois variables et qui contient pour chaque modalité croisée la fréquences de ménages qui sont caractérisés par ces trois modalités,
* la population synthétique est générée en répétant simplement ces fréquences multipliées par la taille  de la population synthétiques. Ces fréquences peuvent donc être vues comme des ipondi
* Modélisatioon du problème : 
  * construction des matrices de contrainte
  * vérification que le système est solvable
  * résolution par l'algorithme de maximisation de l'entropie
* Analyse des résultats en comparant les fréquences initiales du tableau de contingence et les probabilités qui maximisent l'entropie



# Construction de la population synthétique avec son taux de motorisation

## Tableau de contingence


### Initialisation

Il y a 3 modalités pour l'age de la personne de référence , 4 modalités pour la taille du ménage, 2 modalités pour la variable motorisation


In [ ]:
M_a=c('moins_de_30ans', '30-60ans','plus_de_60ans') # Modalite age
M_t=c('1', '2','3','4_et_plus') # Modalite taille
M_m=c('yes','no') # Modalité motorisé ou pas 
na <- length(M_a)
nt <- length(M_t)
nm <- length(M_m)
ndeg = na*nt*nm # nombre de degrés de liberté
nmod = na+nt+nm # nombre de modalités


C=expand.grid(a=M_a,t=M_t,m=M_m,p=0,r=0) # tableau de contingence
n=100000 # N doit être divisible par 10000 pour éviter les erreurs d'arrondi
print(C)


###  Fréquence pour les ménages dont la personne de référence a entre 0 et 30 ans
nous supposons que 20% des ménages ont la  personne de référence  qui a entre 0 et 30 ans,
Ils sont motorisés à 10% quand ils sont seuls
à 20 % quand ils sont deux
à 50% quand ils sont trois
à 60% quand  iks ont quatre


In [ ]:
pa1=0.2 # proportion de personnes ayant l'age 1

C[C$a==M_a[1] & C$t==M_t[1] & C$m==M_m[1],]$p=pa1*0.4*0.1
C[C$a==M_a[1] & C$t==M_t[1] & C$m==M_m[2],]$p=pa1*0.4*0.9
C[C$a==M_a[1] & C$t==M_t[2]  & C$m==M_m[1],]$p=pa1*0.3*0.2
C[C$a==M_a[1] & C$t==M_t[2] & C$m==M_m[2],]$p=pa1*0.3*0.8
C[C$a==M_a[1] & C$t==M_t[3] & C$m==M_m[1],]$p=pa1*0.2*0.5
C[C$a==M_a[1] & C$t==M_t[3] & C$m==M_m[2],]$p=pa1*0.2*0.5
C[C$a==M_a[1] & C$t==M_t[4] & C$m==M_m[1],]$p=pa1*0.1*0.6
C[C$a==M_a[1] & C$t==M_t[4] & C$m==M_m[2],]$p=pa1*0.1*0.4
print(C)


###  Fréquence pour les ménages dont la personne de référence a entre 30 et 60 ans
nous supposons que  70% des ménages ont la personne de référence qui a entre 30 et 60 ans
Ils sont motorisés à 80% quand ils sont seuls
à 90 % quand ils sont deux
à 90% quand ils sont trois
à 90% quand  isls ont quatre


In [ ]:
pa2=0.7 # proportion de personnes ayant l'age 2

C[C$a==M_a[2] & C$t==M_t[1] & C$m==M_m[1],]$p=pa2*0.3*0.80
C[C$a==M_a[2] & C$t==M_t[1] & C$m==M_m[2],]$p=pa2*0.3*0.20
C[C$a==M_a[2] & C$t==M_t[2]  & C$m==M_m[1],]$p=pa2*0.3*0.9
C[C$a==M_a[2] & C$t==M_t[2] & C$m==M_m[2],]$p=pa2*0.3*0.1
C[C$a==M_a[2] & C$t==M_t[3] & C$m==M_m[1],]$p=pa2*0.3*0.9
C[C$a==M_a[2] & C$t==M_t[3] & C$m==M_m[2],]$p=pa2*0.3*0.1
C[C$a==M_a[2] & C$t==M_t[4] & C$m==M_m[1],]$p=pa2*0.1*0.9
C[C$a==M_a[2] & C$t==M_t[4] & C$m==M_m[2],]$p=pa2*0.1*0.1
print(C)


###  Fréquence pour les ménages dont la personne de référence a plus de 60 ans
ils représentent 10% des ménages
Ils sont motorisés à 70% quand ils sont seuls
à 80 % quand ils sont deux
à 80% quand ils sont trois
à 90% quand  ils ont quatre


In [ ]:
pa3=0.1 # proportion de personnes ayant l'age 3

C[C$a==M_a[3] & C$t==M_t[1] & C$m==M_m[1],]$p=pa3*4/10*0.7
C[C$a==M_a[3] & C$t==M_t[1] & C$m==M_m[2],]$p=pa3*4/10*0.3
C[C$a==M_a[3] & C$t==M_t[2] & C$m==M_m[1],]$p=pa3*4/10*0.8
C[C$a==M_a[3] & C$t==M_t[2] & C$m==M_m[2],]$p=pa3*4/10*0.2
C[C$a==M_a[3] & C$t==M_t[3] & C$m==M_m[1],]$p=pa3*1/10*0.8
C[C$a==M_a[3] & C$t==M_t[3] & C$m==M_m[2],]$p=pa3*1/10*0.2
C[C$a==M_a[3] & C$t==M_t[4] & C$m==M_m[1],]$p=pa3*1/10*0.9
C[C$a==M_a[3] & C$t==M_t[4] & C$m==M_m[2],]$p=pa3*1/10*0.1
 print(C)


## Population synthétique

A partir du tableau de contingence, C, la population synthétique est construite en répétant les ménages


In [ ]:
Popp=c();
for (o in M_a) {
  for (oo in M_t){
    for (ooo in M_m){
    nn = round(n*C[C$a==o & C$t==oo & C$m==ooo,]$p,0)
    Poppp=cbind( c(rep(o,nn)), c(rep(oo,nn)), c(rep(ooo,nn)))
    Popp=rbind(Popp, Poppp)
    } # fin for ooo
  } # fin for oo
  } # fin for o
Pop=data.frame(a=as.factor(Popp[,1]),t=as.factor(Popp[,2]),m=as.factor(Popp[,3]) )
#print(Pop) # utilisé en test unitaire pour vérifier que les revenus sont biens fournis.


# Modélisation du problème

## Construction des contraintes

### Contrainte sur les  données agrégées

L'information fournie est une information agrégée sur le taux de motorisation par classe d'age.


In [ ]:
D_agr=c(); # tableau des données agrégées analysable par un humain pour vérifier que le code est correct
M_agr=c() ; # matrice des contraintes sur les probabilités pour la maximisation de l'entropie
I_agr=c() ; # contraintes sur les données agréees pour la maC[C$a==M_a[1] & C$t==M_t[1] & C$m==M_m[1],]$p=pa1*0.4*0.1ximmisation de l'entropie, la contrainte s'écrit M_agr * p = I_agr
motorise=M_m[1]
non_motorise = M_m[2]
for (p in M_a) {
  nn=nrow(Pop[Pop$a==p,]) 
  D_agr=rbind(D_agr,cbind(p,nrow(Pop[Pop$a==p & Pop$m==motorise,])/nn))
  
  Ligne_m=c();
  Ligne_nm=c();
  for (o in M_a) {
    for (oo in M_t){
      for (ooo in M_m){
        if (o==p) {
          if (ooo==motorise) {
             Ligne_m=cbind(Ligne_m,1)
             Ligne_nm=cbind(Ligne_nm,0)
          }
          else {
           Ligne_m=cbind(Ligne_m,0)
           Ligne_nm=cbind(Ligne_nm,1)}}
        else {
             Ligne_m=cbind(Ligne_m,0)
             Ligne_nm=cbind(Ligne_nm,0)
          }
      }# fin ooo
    } # fin oo
  } # fin o
  M_agr=rbind(M_agr, Ligne_m, Ligne_nm);
  probabilite_age_motorise= nrow(Pop[Pop$a==p & Pop$m==motorise,])/n ;
  probabilite_age_non_motorise= nrow(Pop[Pop$a==p & Pop$m==non_motorise,])/n ;
  I_agr=rbind(I_agr, probabilite_age_motorise, probabilite_age_non_motorise)
} # fin p
print(D_agr)


### Construction des contraintes sur les probabilités des modalités croisées age et taille

les autres contraintes sont construites à partir de la population synthétique en calculant les fréquences des modalités croisées entre age et taille


In [ ]:
D_conjointe=c(); # tableau des fréquences des modalités croisés age, taille, analysabe par un humain
M_conjointe=c() ; # matrice des contraintes sur les probabilités des modalités crisées pour la maximisaPtion de l'entropie
I_conjointe=c() ; # contraintes sur les probabilités croisées pour la maximmisation de l'entropie, la contrainte s'écrit M_conjointe * p = I_conjoint
motorise=M_m[1]
non_motorise = M_m[2]
for (p in M_a) {
  for (q in M_t) {
  nn=nrow(Pop[Pop$a==p & Pop$t==q,]) 
  probabilite_modalite_croise=nn/n
  D_conjointe=rbind(D_conjointe,cbind(p,q,probabilite_modalite_croise))
  Ligne=c();
  for (o in M_a) {
    for (oo in M_t){
      for (ooo in M_m){
        if ((o==p) & (oo==q)) {
             Ligne=cbind(Ligne,1)
          }
          else {
           Ligne=cbind(Ligne,0)
          }
      }# fin ooo
    } # fin oo
  } # fin o
  M_conjointe=rbind(M_conjointe, Ligne)
  I_conjointe=rbind(I_conjointe, probabilite_modalite_croise)
} # fin q
} # fin p
print(D_conjointe) # vérification sur cette table est rrrectte


### Aggrégation de toutes les contraintes



In [ ]:
# en prenant toutes les contraintes
Mom = rbind(matrix(rep(1,ndeg),nrow=1,ncol=ndeg), M_agr, M_conjointe)
Eta = rbind(1,I_agr, I_conjointe)


##  Résolution du problème sous la forme de la maximisation de l'entropie



In [ ]:
# vérification du rang de la matrice
A=qr(t(Mom))
I=A$pivot[1:A$rank]


In [ ]:
# vérification que le système est compatible
library('minxent') # minimisation de l'entropie
library(boot) # POV 14 septembre pour avoir la fonction simplexe
M=dim(Mom[I,])
mc=M[2] # nombre de colonne, c'est aussi le nombre de variable
ml=M[1] 
Coef_objectif=c(rep(0,mc), rep(1,ml)) # fonction objectif
Id=diag(rep(1,ml))
M_equation= cbind(Mom[I,], Id)
result=simplex(
a=Coef_objectif,
A3=M_equation, #contraines
b3=Eta[I])
#POV test du 13 octobre 2020  Mom[I,]%*%result$soln[1:264]-Eta[I]
tol_compatibilite <- 1e-8 ; # tolérance sur la compatibilité du système (vérifier )
if (abs(result$value)>tol_compatibilite) {
      print(" système incompatible")
      indice_contrainte_violee=which.max(result$soln[seq(mc+1,mc+ml)]) #voir la contrainte qui déconne le plus (voir dans les 16 dernières, voir ce qui déconne le plus) break ?
      p_out=matrix(result$soln[1:ndeg],nrow=ndeg,ncol=1) # POV VERRUE comme le système est incompatible !
     } else {
  print("système compatible")
  p_adm=matrix(result$soln[1:ndeg],nrow=ndeg,ncol=1) 
  p_init=rep(as.double(1/ndeg), ndeg)
p_out=tryCatch(
        expr = {
           Sortie=minxent.multiple(q=p_init,G=Mom[I,],eta=Eta[I],lambda=c(rep(0, length(I)-1)))  
            p_out=Sortie$Estimates
        },
        error = function(e){
            message('Erreur sur minxent.multiple (maximisation de l entropie) : optimisation sqp  ')
            print(e)
            return()
        },
        warning = function(w){
            message('Caught an warning!')
            print(w)
            return(p_out)
        },
        finally = NULL
    )   # fin de la gestion des erreurs de minxent.multiple 

} # fin si le système est compatible
p_out=matrix(p_out,nrow=length(p_out),ncol=1)
  


# Analyse des résultats



In [ ]:
k=0
  for (o in M_a) {
    for (oo in M_t){
      for (ooo in M_m){
        k=k+1
        C[C$a==o & C$t==oo & C$m==ooo,]$r=p_out[k]
      }# fin ooo
    } # fin oo
  } # fin o
print(C)


In [ ]:
sum(abs(C$p-C$r))




Les probabilités prédites (r) sont relativement proches des  fréquences initiales (p)  pour les grandes fréquences  (celles qui sont supérieure à 5%). Elles peuvent être très éloignées des petites fréquences (inférieurs à 5%). L'erreur en valeur absolue est de 14% ce qui est très raisonnable compte tenu du fait que la solution initiale est la probabilité uniforme valant  0.04 (= 1/ nombre de degrés de liberté qui est égal à 24).  Cette probabilité est correctement déformé pour s'adapter aux contraintes.
